In [2]:
import networkx as nx
from virtuoso import *
from config import NODE_TYPE
import math
from solr import get_info_solr
GRAPH = "http://opendata.aragon.es/def/ei2a"

In [3]:

# Get the name of a concept acording to config file
def get_name(concept,node_type):
    if node_type and "rawName" in node_type and node_type["rawName"]:
        return node_type["rawName"]
    elif node_type and "name" in node_type and node_type["name"]:
        query = get_prefixes()
        query += ('SELECT DISTINCT ?name '
                  'FROM <' + GRAPH + '> '
                  'WHERE {{{0} ')
        query += replace_prefixes(node_type["name"])
        query += " ?name }}"
        query = query.format(convert_concept(concept))
        data = sparqlQuery(query,VIRTUOSO_URL)
        name = ""
        for row in data:
            if row["name"]["value"] != "http://www.w3.org/2002/07/owl#NamedIndividual":
                name = row["name"]["value"]
        if "#" in name:
            name = name.split("#")[1]
        return name
    else:
        return concept

# Get the type of a concept acording to config file
def get_type(concept):
    for prefix in PREFIXES:
        if prefix[1] in concept:
            query = get_prefixes()
            query +=(
                'SELECT DISTINCT ?type '
                'FROM <' + GRAPH + '> '
                'WHERE{{'
                '{0} rdf:type owl:NamedIndividual . '
                '{0} rdf:type ?type FILTER (?type not in (owl:NamedIndividual))}}')
            concept = replace_prefixes(concept)
            query = query.format(concept)
            data = sparqlQuery(query, VIRTUOSO_URL)
            typeToCmp = ""
            for row in data:
                typeToCmp = row["type"]["value"]
            for node_type in NODE_TYPE:
                if node_type["identifier"] in typeToCmp:
                    return node_type

# Get the information of a concept acording to config file
def get_info(concept,node_type):
    if node_type and node_type["properties"]:       # Procesar solo si  tiene alguna propiedad
        query = get_prefixes()
        query += "SELECT DISTINCT "
        properties = [replace_prefixes(x) for x in node_type["properties"]]
        for i,property in enumerate(properties):
            varialbeProperty = get_variable_name(property,i)
            query += " ?" + varialbeProperty + " ?label" + varialbeProperty
        query += " FROM <" + GRAPH + "> WHERE{{"

        for i,property in enumerate(properties):
            varialbeProperty = get_variable_name(property, i)
            query += " {0} " + property + " ?" + varialbeProperty + " FILTER (?" + varialbeProperty + " not in (owl:NamedIndividual)). OPTIONAL{{" + property + ' rdfs:comment ?label' + varialbeProperty + ' FILTER(LANGMATCHES(LANG(?label' + varialbeProperty + '), "es"))}}.'
        query += "}}"
        concept = replace_prefixes(concept)
        query = query.format(concept)
        data = sparqlQuery(query, VIRTUOSO_URL)
        retList = {}
        if data:
            for i, property in enumerate(properties):
                varialbeProperty = get_variable_name(property, i)
                key = data[0].get("label" + varialbeProperty, {}).get("value", varialbeProperty)
                retList[key] = data[0][varialbeProperty]["value"]
            return retList

# Build a graph with the result of the query to Virtuoso
def build_graph(data,concept,node_type_central,labelCentral,labelFrom,labelTo,g,level,xCentral,yCentral,centerRecursive):
    if level != 0:
        data = [triple for triple in data if triple.get(labelFrom,{}).get("value") != centerRecursive and triple.get(labelTo,{}).get("value") != centerRecursive and triple.get(labelCentral,{}).get("value") != centerRecursive]
    initialAngle = 2 * math.pi if level == 0 else math.pi * 0.25
    displacement = 0 if level == 0 else math.atan2(yCentral,xCentral)
    for i,triple in enumerate(data):
        radio = (i / len(data)) *initialAngle + displacement
        x = math.cos(radio) * 150 + xCentral
        y = math.sin(radio) * 150 + yCentral
        if labelFrom in triple:
            name = triple[labelFrom]["value"]
            type = get_type(name)
            g.add_node(name,attr_dict={"node_type":type,"x":x,"y":y})
            g.add_edge(name, triple[labelCentral]["value"], label="Entrante")
        else:  # elif labelTo in triple:
            name = triple[labelTo]["value"]
            type = get_type(name)
            g.add_node(name, attr_dict={"node_type":type,"x":x,"y":y})
            g.add_edge(triple[labelCentral]["value"], name, label="Saliente")
        if type and "rawName" in type:  # Apply recursion
            query = build_query(name, type, 20)
            dataRecursive = sparqlQuery(query, VIRTUOSO_URL)
            build_graph(dataRecursive, name, type, "callret-0", "in", "out", g, level + 1, x, y, concept)
    g.add_node(concept, attr_dict={"node_type": node_type_central, "x": xCentral, "y": yCentral})
    return g

# Get the image of a concept acording to config file
def get_image(node_type):
    if node_type and "image" in node_type:
        return node_type["image"]
    else:
        return ""

# Build the nodes section of the json
def fill_nodes(nodes):
    retList = []
    for j,node in enumerate(nodes):
        nodeName = node[0]
        node_type = node[1]["attr_dict"]["node_type"]
        name = get_name(nodeName,node_type)
        image = get_image(node_type)
        retList.append({
            "label": name,
            "nodeName": nodeName,
            "fin_x": node[1]["attr_dict"]["x"],
            "fin_y": node[1]["attr_dict"]["y"],
            "id": j,
            "image": {
                "url": image
            }
        })
    return retList

# Build the edges section of the json
def fill_edges(nodes,edges):
    i = 0
    aux = []
    for (source, target) in edges:
        aux.append({
            "source": list(nodes).index(source),
            "target": list(nodes).index(target),
            "id": i
        })
        i += 1
    return aux

# Build the query acording to te config file
def build_query(concept, node_type, max_nodes):
    query = ""
    query += get_prefixes()
    concept = convert_concept(concept)
    if not node_type:  # En caso de fallo
        query += ('SELECT DISTINCT {0} ?in ?preIn '
                 'FROM <' + GRAPH + '> '
                 'WHERE {{ ?in ?preIn {0} }}')
    else:  # Si es una instancia
        query += ('SELECT DISTINCT {0} ?in ?preIn ?preOut ?out '
                 'FROM <' + GRAPH + '> '
                 'WHERE {{'
                 '{{ ?in ?preIn {0} }}'
                 'UNION'
                 '{{ {0} ?preOut ?out }}'
                 'FILTER ( ?preIn in (')
        query += ",".join(convert_concept(edge) for edge in node_type["incomingEdges"])
        query += ") || ?preOut in ("
        query += ",".join(convert_concept(edge) for edge in node_type["outgoingEdges"])
        query += "))}} LIMIT " + str(max_nodes)
    query = query.format(concept)
    return query

# Main function, build the json
def build_json(concept, max_nodes):
    retList = {
        "graph":{
            "edges": [],
            "nodes": []
        },
        "info":{},
        "alertMaximumNodes":None
    }

    node_type = get_type(concept)

    query = build_query(concept, node_type, max_nodes)

    data = sparqlQuery(query, VIRTUOSO_URL)
    retList["alertMaximumNodes"] = len(data) == int(max_nodes)

    g = nx.DiGraph()
    build_graph(data, concept, node_type, "callret-0", "in", "out", g, 0, 0, 0, "")

    retList["graph"]["nodes"] = fill_nodes(g.nodes(data=True))

    retList["graph"]["edges"] = fill_edges(g.nodes(),g.edges())

    retList["info"] = get_info(concept,node_type)

    if(node_type and node_type["identifier"] == "http://opendata.aragon.es/def/ei2a#WebPage"):
        retList["solrInfo"] = get_info_solr(get_name(concept,node_type))

    return retList


In [4]:
REQUEST=json.dumps({
    'body': {'concept':'http://opendata.aragon.es/def/ei2a#Entidad420','nodes':20}})
print(REQUEST)

{"body": {"concept": "http://opendata.aragon.es/def/ei2a#Entidad420", "nodes": 20}}


In [5]:
# POST /entity
req = json.loads(REQUEST)
concept = req['body']['concept']
max_nodes = req['body']['nodes']

json_response = build_json(concept, max_nodes)

status=200
print (json.dumps(json_response))

{"graph": {"edges": [{"source": 0, "target": 1, "id": 0}, {"source": 0, "target": 2, "id": 1}, {"source": 3, "target": 1, "id": 2}, {"source": 3, "target": 4, "id": 3}, {"source": 5, "target": 1, "id": 4}, {"source": 5, "target": 6, "id": 5}, {"source": 7, "target": 1, "id": 6}, {"source": 7, "target": 8, "id": 7}, {"source": 9, "target": 1, "id": 8}, {"source": 9, "target": 10, "id": 9}, {"source": 11, "target": 1, "id": 10}, {"source": 11, "target": 12, "id": 11}, {"source": 13, "target": 1, "id": 12}, {"source": 13, "target": 14, "id": 13}, {"source": 15, "target": 1, "id": 14}, {"source": 15, "target": 16, "id": 15}, {"source": 17, "target": 1, "id": 16}, {"source": 17, "target": 18, "id": 17}, {"source": 19, "target": 1, "id": 18}, {"source": 19, "target": 20, "id": 19}, {"source": 21, "target": 1, "id": 20}, {"source": 21, "target": 22, "id": 21}, {"source": 23, "target": 1, "id": 22}, {"source": 23, "target": 24, "id": 23}, {"source": 25, "target": 1, "id": 24}, {"source": 25, "

In [6]:
# ResponseInfo POST /entity
print(json.dumps({
    "status" : status,
    "headers" : {
        "Content-Type" : "application/json"
    }
}))

{"status": 200, "headers": {"Content-Type": "application/json"}}
